# Description

The notebook demonstrates how open-source solvers solve the DaoCross problem.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
from typing import Tuple

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint

In [3]:
try:
    import pulp
except ImportError:
    !sudo /bin/bash -c "(source /venv/bin/activate; pip install pulp)"
    import pulp

In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-32bf17d0-fc88-469c-a1e3-ab9bba90edf5.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.1' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='SorrTask80_dao_cross_optimization'
  hash='67b9c31e9'
  # Last commits:
    * 67b9c31e9 PomazkinG checkpoint                                                        (  15 hours ago) Thu Mar 30 20:02:58 2023  (HEAD -> SorrTask80_dao_cross_optimization, origin/SorrTask80_dao_cross_optimization)
    * 1b60b0b08 PomazkinG checkpoint                                                        (  15 hours ago) Thu Mar 30 19:54:07 2023  

# Order class

In [5]:
# TODO(Grisha): move to a lib and re-use everywhere.
class Order:

    # TODO(Grisha): add type hints, add assertions.
    def __init__(self, action, quantity, base_token, limit_price, quote_token):
        self.action = action
        self.quantity = quantity
        self.base_token = base_token
        self.limit_price = limit_price
        self.quote_token = quote_token

    def __repr__(self):
        return str(self)

    def __str__(self):
        ret = (
            "action=%s, quantity=%s, base_token=%s, limit_price=%s, quote_token=%s"
            % (
                self.action,
                self.quantity,
                self.base_token,
                self.limit_price,
                self.quote_token,
            )
        )
        return ret

# Functions

In [6]:
# TODO(Grisha): consider extending for n orders.
# TODO(Grisha): move to a lib, add unit tests.
def optimize_for_volume(
    order_1: Order, order_2: Order, exchange_rate: float
) -> None:
    """
    Find the maximum transacted volume given the orders and the constraints.

    :param order_1: input buy order
    :param order_2: input sell order
    :param exchange_rate: price of base token / price of quote token
    :return: solver's output in a human readable format
    """
    # Assume the fixed directions.
    hdbg.dassert_eq(order_1.action, "buy")
    hdbg.dassert_eq(order_2.action, "sell")
    #
    hdbg.dassert_lt(0, exchange_rate)
    # Initialize the model.
    problem = pulp.LpProblem("The DaoCross problem", pulp.LpMaximize)
    # Specify the vars. By setting the lower bound to zero it is safe
    # to omit the >= 0 constraint on the executed quantity.
    q_base_asterisk_1 = pulp.LpVariable("q_base_asterisk_1", lowBound=0)
    q_base_asterisk_2 = pulp.LpVariable("q_base_asterisk_2", lowBound=0)
    # Objective function.
    # TODO(Grisha): since the base token is the same, i.e. BTC it's
    # ok to use quantity, however the objective function should be
    # modified to account for different base tokens.
    problem += q_base_asterisk_1 + q_base_asterisk_2
    # Constraints.
    # Random number that is big enough to use the
    # "Big M" method.
    M = 1e6
    # TODO(Grisha): this should be a function of action.
    limit_price_cond_1 = int(exchange_rate <= order_1.limit_price)
    _LOG.info("limit_price_cond_1 is %s", limit_price_cond_1)
    limit_price_cond_2 = int(exchange_rate >= order_2.limit_price)
    _LOG.info("limit_price_cond_2 is %s", limit_price_cond_2)
    # Executed quantity is not greater than the requested quantity
    # given that the limit price condition is satisfied.
    problem += q_base_asterisk_1 <= order_1.quantity + M * (1 - limit_price_cond_1)
    problem += q_base_asterisk_2 <= order_2.quantity + M * (1 - limit_price_cond_2)
    # Executed quantity is zero if the limit price condition is not met.
    problem += q_base_asterisk_1 <= M * limit_price_cond_1
    problem += q_base_asterisk_1 >= -M * limit_price_cond_1
    #
    problem += q_base_asterisk_2 <= M * limit_price_cond_2
    problem += q_base_asterisk_2 >= -M * limit_price_cond_2
    # The number of sold tokens must match the number of bought tokens.
    problem += q_base_asterisk_1 == q_base_asterisk_2
    # Use the default solver and suppress the solver's log.
    solver = pulp.getSolver("PULP_CBC_CMD", msg=0)
    problem.solve(solver)
    # Display the results.
    # TODO(Grisha): probably package the output in a dict.
    _LOG.info(
        "The status is: %s"
        "\nThe total volume (in BTC) exchanged is: %s"
        "\nThe value of exchanged base token from order 1: %s"
        "\nThe value of exchanged base token from order 2: %s"
        "\nThe solution time (in seconds) is: %s",
        pulp.LpStatus[problem.status],
        pulp.value(problem.objective),
        q_base_asterisk_1.varValue,
        q_base_asterisk_2.varValue,
        # TODO(Grisha): double-check that time is in seconds.
        round(problem.solutionTime, 2),
    )


def get_test_orders(
    limit_price_1: float, limit_price_2: float
) -> Tuple[Order, Order]:
    """
    Get toy orders to demonstrate how the solver works.
    
    :param limit_price_1: limit price for the buy order
    :param limit_price_2: limit price for the sell order
    :return: buy and sell orders
    """
    # Genereate buy order.
    action = "buy"
    quantity = 5
    base_token = "BTC"
    quote_token = "ETH"
    order_1 = Order(action, quantity, base_token, limit_price_1, quote_token)
    _LOG.info("Buy order: %s", str(order_1))
    # Generate sell order.
    action = "sell"
    quantity = 6
    base_token = "BTC"
    quote_token = "ETH"
    order_2 = Order(action, quantity, base_token, limit_price_2, quote_token)
    _LOG.info("Sell order: %s", str(order_2))
    return order_1, order_2

# Solve the optimization problem

Any simulation for which the limit price constraint is not satisfied for at least one order ends with no trades being executed.
While if the limit price constraint is satisfied for all orders the trade is executed using the maximum quantity of the base token taking into account the constraint saying that quantity of sold token = quantity of bought token.

In [7]:
exchange_rate = 4
_LOG.info("Exchange rate=%s", exchange_rate)

INFO  Exchange rate=4


## Limit price condition is met for both orders

In [8]:
limit_price_1 = 5
limit_price_2 = 3
test_orders_1 = get_test_orders(limit_price_1, limit_price_2)
optimize_for_volume(test_orders_1[0], test_orders_1[1], exchange_rate)

INFO  Buy order: action=buy, quantity=5, base_token=BTC, limit_price=5, quote_token=ETH
INFO  Sell order: action=sell, quantity=6, base_token=BTC, limit_price=3, quote_token=ETH
INFO  limit_price_cond_1 is 1
INFO  limit_price_cond_2 is 1
INFO  The status is: Optimal
The total volume (in BTC) exchanged is: 10.0
The value of exchanged base token from order 1: 5.0
The value of exchanged base token from order 2: 5.0
The solution time (in seconds) is: 0.02


/venv/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Limit price condition is met only for 1 order

In [9]:
limit_price_1 = 5
limit_price_2 = 5
test_orders_1 = get_test_orders(limit_price_1, limit_price_2)
optimize_for_volume(test_orders_1[0], test_orders_1[1], exchange_rate)

INFO  Buy order: action=buy, quantity=5, base_token=BTC, limit_price=5, quote_token=ETH
INFO  Sell order: action=sell, quantity=6, base_token=BTC, limit_price=5, quote_token=ETH
INFO  limit_price_cond_1 is 1
INFO  limit_price_cond_2 is 0
INFO  The status is: Optimal
The total volume (in BTC) exchanged is: 0.0
The value of exchanged base token from order 1: 0.0
The value of exchanged base token from order 2: 0.0
The solution time (in seconds) is: 0.02


In [10]:
limit_price_1 = 3
limit_price_2 = 3
test_orders_1 = get_test_orders(limit_price_1, limit_price_2)
optimize_for_volume(test_orders_1[0], test_orders_1[1], exchange_rate)

INFO  Buy order: action=buy, quantity=5, base_token=BTC, limit_price=3, quote_token=ETH
INFO  Sell order: action=sell, quantity=6, base_token=BTC, limit_price=3, quote_token=ETH
INFO  limit_price_cond_1 is 0
INFO  limit_price_cond_2 is 1
INFO  The status is: Optimal
The total volume (in BTC) exchanged is: 0.0
The value of exchanged base token from order 1: 0.0
The value of exchanged base token from order 2: 0.0
The solution time (in seconds) is: 0.03


## Limit price condition is not met for both orders

In [11]:
limit_price_1 = 3
limit_price_2 = 5
test_orders_1 = get_test_orders(limit_price_1, limit_price_2)
optimize_for_volume(test_orders_1[0], test_orders_1[1], exchange_rate)

INFO  Buy order: action=buy, quantity=5, base_token=BTC, limit_price=3, quote_token=ETH
INFO  Sell order: action=sell, quantity=6, base_token=BTC, limit_price=5, quote_token=ETH
INFO  limit_price_cond_1 is 0
INFO  limit_price_cond_2 is 0
INFO  The status is: Optimal
The total volume (in BTC) exchanged is: 0.0
The value of exchanged base token from order 1: 0.0
The value of exchanged base token from order 2: 0.0
The solution time (in seconds) is: 0.02
